# Notebook 1: Getting access token

### Getting started:
- Play around with the Graph API Explorer and try to get the information from an user. 
- Due to facebook's privacy policy, you can't get data from other userd. It has to be the your own data. 
- First, registere an access tokens that are already provided and select what kind of endpoints that we want. Then, try a simple command such as the GET method "me?fields=id,name" and it returns the username in Facebook. By playing around with the Graph API Explorer will help learning how to get the syntax for requesting information from Facebook.

### Obtaining a long-lived access token for Facebook's Graph API
The Facebook Graph API uses OAUTH 2.0. The procudure is similar to the OAUTH Dance we used in the 'oath_guide' for Google Drives API, with some changes explained through the steps below. 

In this project overall, the hardest part was to get the access token. We figured that there are four things that needed which were client id, client secret, redirect uri, and authorization url. In order to create request from Facebook, we had to sign up and create an App in Facebook for Developer. Then, we were able to obtain the client id and client secret. The authorization url was a bit tricky but we were able to find it online. The hard part was redirect uri, because it was impossible to find it online. We spent large amount of time on figuring what the redirect uri was. Eventually, we decided to have our personal facebook page as our redirect uri. After putting all the information in the keychain, we were able to get the access token.

In [8]:
# Importing all the required Libraries
import importlib
import keys_project
importlib.reload(keys_project)
import json
import webbrowser
import pprint
import requests
keychain = keys_project.keychain
from requests_oauthlib import OAuth2Session
from requests_oauthlib.compliance_fixes import facebook_compliance_fix

# Setting the client_id, scope, redirect and client_secret from the keychain
client_id = keychain['facebook']['client_id']
scope = keychain['facebook']['scope']
redirect = keychain['facebook']['redirect_uri']
client_secret = keychain['facebook']['client_secret']

After deciding which redirect uri was going to be, it was a lot easier to finish up the first notebook. We did not have to spend a lot of time on getting the access token later on. In the cell above, we were obtaining the required libraries and also got client information (stored in keychain) which was obtained after registering an application with Facebook Developers. The 'scope' variable was a list of strings which contain a list of permissions to access different kinds of user data such as 'location', 'posts', 'education', 'photos', 'likes', 'reactions', 'friends', and so on. The list of possible scopes can be found on the 'permissions' page of the Graph API documentation:

https://developers.facebook.com/docs/facebook-login/permissions

When we worked with the Google API in class, the redirect URL was downloaded with other client information. But for Facebook, we had to figure out what the redirect URL was. We tried several redirects such as the Facebook Login page, a redirect back to our application and a few more. But we found to get a 'code' to exchange for a access token, our redirect would need to be a URL to a user's facebook profile.

In [9]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['facebook']['auth_uri'])
webbrowser.open(auth_url) #opening authorization url in the webrowser

True

Once redirected, the 'code' was found as part of the URL in the web-browser. We then copy pasted that into a variable below and then used it to exchange it for a short-lived token. The code changes everytime we opened authorization url in the web brower. Therefore, we need to copy the new code in the url and use it.

In [10]:
code="AQA83i8e2oDu99kLJk7dzVLXrehPEsQ0gJ5DtOGfIB8bRhKiE0Nm3EHfyhUuBG7pQQ5i-blwqRRIJ8DVXQ67KUL4o2wwBHzauuHnLL1n-7j_KztgI_wfPTzI378_f2qwjHu6mvQYaG-QNMkhb_l6WW2IlliA8Vu76uBtoxcI0TMYBgILn0qeFINe_E_THHCoIVWbkO9Z46nW9qv9SCwL-eOkgMsmcA3HKtl1Ad3rq9JMTj0MvNH_bjrIe08UP-beE1l9vdVGngM8narF-17UzQ7QwPbzOAZteW9LUKUvvneEMod6X7yTg91Sbow5EkNF_24"
token = session.fetch_token(keychain['facebook']['token_uri'], 
                                   code=code, 
                                   client_secret=keychain['facebook']['client_secret']) #fetching the access token

Once we have got the short-lives token which lasts only for a couple of hours, we must exchange it for a long-lived token which lasts for about 60 days. Facebook does not provide a way to refresh their tokens once expired. To get a new token the login flow must be followed again to obtain a short-lived token which needs to be exhanged, once again, for a long lived token. This is expressed in the facebook documentations as follows:

"Even the long-lived access token will eventually expire. At any point, you can generate a new long-lived token by sending the person back to the login flow used by your web app - note that the person will not actually need to login again, they have already authorized your app, so they will immediately redirect back to your app from the login flow with a refreshed token

After doing the above you will obtain a new short-lived token and then you need to perform the same exchange for a long-lived token ....
"

In order to get a long-lived token, we followed the documentation on the facebook page:

https://developers.facebook.com/docs/facebook-login/access-tokens/expiration-and-extension

To get this token, we making a GET request to the Facebook Graph API with header 4 pieces of information 'client-secret', 'client_id', 'grant_type' and 'fb_exchange_token' which is the short lived token to be exchanged. 

In [11]:
#This code exchanges a short-lived token for a long-lived token using a get request with 
#parameters specified in payload 

payload={"grant_type":"fb_exchange_token","client_secret":keychain['facebook']['client_secret']\
       ,"client_id":keychain['facebook']['client_id'],"fb_exchange_token":token['access_token']}

token_long_temp=session.get("https://graph.facebook.com/oauth/access_token?",params=payload)
#converting the response to json
string_token = token_long_temp.content.decode('utf-8')
token_long = json.loads(string_token)
token_long

{'access_token': 'EAAdjsZA7XHYoBAMTj4BS2mrqQlnW50OX2h5Fx47b5JpEr6LjYxgHaBoQFWDh7jJl4Y7WmnMX9QUPZCgYXZB7Y7YPyysRg91ELNO6nXvIC9VRg9Pu6ziSdSemZCjIpfflTSxm4e8vmuKCaW7xg3ZBlybvZAdqmkOZCebvAZB618XzxAZDZD',
 'token_type': 'bearer'}

### Conclusion
Now that we have obtained a long-lived token, we are good to start obtaining data through API calls. We have put the access token in the keychain dictionar as keychain['facebook']['access_token'] to import into Notebook_2. At this point, the first notebook is finished. We started writing this journal right after we finish up debuging and testing the code. To sum up, this notebook took most of our time when doing this project. It was mainly because of the fact that we were struggling with finding the redirect uri for the access token. As soon as we figured out what the redirect uri was, we were able to finish up this notebook and spend time on the other two. Overall, it was challanging, but it was really fun. Getting the access token taught me how to research and self-explore.